# Clothing Image Classifier 
This notbook I build a clothing image classifier using A **fully connected nueral network** and the FashionMNIST dataset

In [ ]:
# imports 
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

## Class appraoch 

In [ ]:
# create transform 
transform = transforms.Compose([
    transforms.ToTensor(),      # convert to tensor 
    transforms.Normalize((0.5), (0.5))      # center data around 0 for faster and stable learning  
                                            # This normalizes the single chanel being worked with
])

Normalize((0.5,), (0.5,)) line performs this math:$$output = \frac{input - 0.5}{0.5}$$
- If the pixel is 0.0 (black): $(0.0 - 0.5) / 0.5 = \mathbf{-1.0}$
- If the pixel is 1.0 (white): $(1.0 - 0.5) / 0.5 = \mathbf{1.0}$

Data is now centered around 0.

## Create Datasets and Data Loaders

In [ ]:
# datasets
trainset = torchvision.datasets.FashionMNIST('./data', download=True, train=True, 
                                             transform=transform)
testset = torchvision.datasets.FashionMNIST('./data', download=True, train=False, 
                                            transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=30, 
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=30, 
                                         shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

## Classes 

In [ ]:
# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

## Check DataLoader Output 

In [ ]:
# show img function 

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # (C, H, W) -> (H, W, C)

# get image batch from trai data 
dataiter = iter(trainloader)
images, labels = next(dataiter)

# check the shape 
print(f'Batch shape: {images.shape}') # should be [30, 1, 28, 28] [batch_size, channels, H, W]

# show images and labels
imshow(torchvision.utils.make_grid(images))
print(' | '.join(f'{classes[labels[i]]}' for i in range(len(labels))))

## Create Model 

In [ ]:
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        # linear layers
        self.fc1 = nn.Linear(28 * 28, 1024)
        self.fc2 = nn.Linear(1024, 10) # output layer
    
    def forward(self, x): 
        # pass the image through Fully connected layers
        
        # flatten
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

fcn = FCN()    

## Loss function and Back Prop. Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(fcn.parameters(), lr=0.001, momentum=0)

## Train Model 

In [ ]:
for epoch in range(30):
    running_loss = 0.0 
    print_interval = 400
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data
        optimizer.zero_grad()
        
        # forward pass
        outputs = fcn(inputs)
        loss = criterion(outputs, labels)
        
        # # back prop
        loss.backward()
        optimizer.step()
        
        # print running stats
        running_loss += loss.item()
        if i % print_interval == print_interval - 1:
            avg_loss = running_loss / print_interval
            print(f'[Epoch {epoch + 1}, Batch {i + 1:5d}] loss: {avg_loss:.3f}')
            running_loss = 0.0 

# Test Model 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        
        images, labels = data
        
        # raw model scores
        outputs = fcn(images)
        # extract highest score
        _, predicted = torch.max(outputs.data, 1)
        # count total images processes so far
        total += labels.size(0)
        # count how many in this batch matches the labels
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total 
print(f'Final accuracy on {total} test images: {accuracy:.1f}%')